In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("/Users/tyleremanuel/Bench/BDB Code-Data/all_weeks_miss.csv")

In [4]:
data.columns

Index(['Unnamed: 0', 'gameId', 'playId', 'nflId', 'displayName', 'frameId',
       'time', 'jerseyNumber', 'club', 'playDirection', 'x', 'y', 's', 'a',
       'dis', 'o', 'dir', 'event', 'ballCarrierId', 'ballCarrierDisplayName',
       'playDescription', 'quarter', 'down', 'yardsToGo', 'possessionTeam',
       'defensiveTeam', 'gameClock', 'preSnapHomeScore', 'preSnapVisitorScore',
       'playResult', 'expectedPoints', 'expectedPointsAdded', 'tackle',
       'assist', 'forcedFumble', 'pff_missedTackle'],
      dtype='object')

In [5]:
temp = data.loc[data['pff_missedTackle'] == 1]
missed_data = temp.loc[temp['frameId'] == 1]

In [6]:
missed_data

,Unnamed: 0,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,...,gameClock,preSnapHomeScore,preSnapVisitorScore,playResult,expectedPoints,expectedPointsAdded,tackle,assist,forcedFumble,pff_missedTackle
23252,23252,2022090800,1102,52492.0,Terrell Lewis,1.0,2022-09-08 21:03:51.299999,52.0,LA,right,...,9:37,0.0,7.0,2.0,3.894043,-0.132479,0.0,0.0,0.0,1.0
28764,28764,2022090800,1385,40107.0,Micah Hyde,1.0,2022-09-08 21:17:46.900000,23.0,BUF,left,...,4:14,0.0,10.0,18.0,4.027252,1.463392,0.0,0.0,0.0,1.0
29676,29676,2022090800,1385,46232.0,Tim Settle,1.0,2022-09-08 21:17:46.900000,99.0,BUF,left,...,4:14,0.0,10.0,18.0,4.027252,1.463392,0.0,0.0,0.0,1.0
42160,42160,2022090800,2163,43335.0,A'Shawn Robinson,1.0,2022-09-08 22:00:42.299999,94.0,LA,left,...,10:07,10.0,10.0,13.0,3.941324,0.789176,0.0,0.0,0.0,1.0
44440,44440,2022090800,2208,42816.0,Troy Hill,1.0,2022-09-08 22:02:06.900000,2.0,LA,left,...,8:45,10.0,10.0,8.0,5.137182,0.491406,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11833089,11833089,2022110700,2965,43409.0,David Onyemata,1.0,2022-11-07 22:32:22.200000,93.0,NO,left,...,11:07,6.0,17.0,17.0,3.054931,1.391480,0.0,0.0,0.0,1.0
11834002,11834002,2022110700,2965,52665.0,Malcolm Roach,1.0,2022-11-07 22:32:22.200000,97.0,NO,left,...,11:07,6.0,17.0,17.0,3.054931,1.391480,0.0,0.0,0.0,1.0
11837120,11837120,2022110700,3037,52482.0,Zack Baun,1.0,2022-11-07 22:34:45.000000,53.0,NO,left,...,9:36,6.0,17.0,-4.0,3.964865,-1.017409,0.0,0.0,0.0,1.0
11839991,11839991,2022110700,3174,48027.0,Kaden Elliss,1.0,2022-11-07 22:41:57.099999,55.0,NO,left,...,7:47,6.0,20.0,5.0,5.512065,0.399304,0.0,0.0,0.0,1.0


In [7]:
# (gameId, playId, displayName)
missed_tackler_data = []
for index, row in missed_data.iterrows():
    missed_tackler_data.append((row['gameId'], row['playId'], row['displayName']))

In [8]:
missed_tackler_data[:1]

[(2022090800, 1102, 'Terrell Lewis')]

In [40]:
missed_tackle_frame_data = []
for combo in missed_tackler_data:
    tackler_frames = data.loc[(data['gameId'] == combo[0]) & (data['playId'] == combo[1]) & (data['displayName'] == combo[2])]
    runner_frames = data.loc[(data['gameId'] == combo[0]) & (data['playId'] == combo[1]) & (data['displayName'] == tackler_frames.iloc[0, :]['ballCarrierDisplayName'])]
    closest_frame = 0
    min_distance = 1e9
    for i in range(len(tackler_frames)):
        tackler_frame = tackler_frames.iloc[i, :]
        runner_frame = runner_frames.iloc[i, :]
        tackler_x = tackler_frame['x']
        tackler_y = tackler_frame['y']
        runner_x = runner_frame['x']
        runner_y = runner_frame['y']
        distance = (((tackler_x-runner_x) ** 2) + (tackler_y - runner_y) ** 2) ** 0.5
        if distance < min_distance:
            min_distance = distance
            closest_frame = i
    missed_tackle_frame_data.append((combo[0], combo[1], closest_frame))

print(missed_tackle_frame_data)

[(2022090800, 1102, 13), (2022090800, 1385, 56), (2022090800, 1385, 38), (2022090800, 2163, 42), (2022090800, 2208, 13), (2022090800, 2336, 11), (2022090800, 2712, 34), (2022090800, 2839, 58), (2022090800, 2958, 33), (2022090800, 3576, 41), (2022090800, 3576, 19), (2022091100, 78, 65), (2022091100, 322, 11), (2022091100, 395, 28), (2022091100, 546, 84), (2022091100, 1067, 27), (2022091100, 1191, 13), (2022091100, 1485, 5), (2022091100, 2069, 29), (2022091100, 2135, 20), (2022091100, 2135, 11), (2022091100, 2332, 48), (2022091100, 2491, 39), (2022091100, 3926, 41), (2022091101, 296, 30), (2022091101, 641, 40), (2022091101, 641, 42), (2022091101, 1213, 37), (2022091101, 1564, 59), (2022091101, 1590, 52), (2022091101, 1616, 31), (2022091101, 1691, 12), (2022091101, 1980, 29), (2022091101, 2001, 33), (2022091101, 2365, 32), (2022091101, 2365, 48), (2022091101, 2407, 11), (2022091101, 3080, 46), (2022091101, 3080, 33), (2022091101, 3101, 23), (2022091101, 3287, 47), (2022091101, 3287, 32), 

In [41]:
missed_tackles = pd.DataFrame(missed_tackle_frame_data, columns = ['gameId', 'playId', 'frameId'])

In [1]:
for index, row in data[:1000].iterrows():
    if row['playId'] == missed_tackles['playId'][index]:
        print(row)

NameError: name 'data' is not defined